In [2]:
from inventory import *
import requests
from bs4 import BeautifulSoup
from PIL import Image
import os.path
from os import path
from random import randrange
import numpy as np
import ast

In [4]:
# get links for all mezlan shoes
links = []
types = ['formal-dress-shoes', 'loafers', 'oxford', 'boots', 'sneakers']

for t in types:
    i = 1
    print(t)
    while True:
        result = requests.get("https://www.carrucci.com/collections/" + t + "?page="+str(i))
        print(i)
        i += 1
        soup = BeautifulSoup(result.content, "html.parser")
        a = soup.find_all("div", {"class": "ProductItem__Wrapper"})
        ln = ["https://www.carrucci.com" + str(ax.find('a')['href']) for ax in a]
        if len(ln) == 0:
            break
        links.extend(ln)
    print()

formal-dress-shoes
1
2

loafers
1
2
3
4
5

oxford
1
2
3
4

boots
1
2

sneakers
1
2



In [6]:
r = {}
for l in links:
    result = requests.get(l)
    soup = BeautifulSoup(result.content, "html.parser")
    title = soup.title.text.replace("\n", "").replace("  ", "").replace(" – Carrucci Shoes", "").replace("Carrucci ", "")
    r[title] = soup
    print(title)

Genuine Leather Bow Tie Loafer
KS525-210SP Bow Tie Loafer
KS525-103V Velvet Loafer
KS308-101V Velvet Buckle Loafer
KS805-10SS Studs Loafer-Black
KS525-102V, Velvet Prom Loafer-Emerald
KS525-01P, Patent Leather Lace-up Oxford
KS308-01 Velvet Loafer
Prom Loafer in Leather Sole
KS805-05SG Suede Studs Loafer
KS805-07SV Hand Embroidered Loafer
KS525-210N Formal Dress Shoes
KS525-102P Patent Leather Bow Tie Dress Shoes
KS503-37A Hand Painted Double Monk Loafer
KS525-17S Printed Suede Double Monk Straps PRE ORDER
KS478-505 Leather Buckle Loafer
KS515-04 Double Monk Strap Leather Loafer
KS503-02 Signature Buckle Loafer
KS479-601 Contrast Penny Loafer
KS525-305 Soft Leather Casual Buckle Loafer/Mule
KS515-03 Whole Cut Leather Loafer
KS509-23 Double Monk Straps Leather Shoes
KS478-120SC Patina Finish Leather Penny Loafer
KS478-119S Suede Penny Loafer w/ Leather Trim
KS503-60 Cross Strap Leather Loafer
KS500-12 Woven Cap Toe Tassel Loafer
KS478-118S Leather Suede Penny Loafer
KS478-503 Leather Pe

In [15]:
def getType(soup):
    return "type"

In [127]:
def get(soup):
    title = soup.title.text.replace("\n", "").replace("  ", "").replace(" – Carrucci Shoes", "")
#     print(title)

for i in r.keys():
    get(r[i])

soups = [i for i in r.values()]

for soup in soups:
    # soup = soups[1]
    
    # vendor
    vendor = "Carrucci"

    # title

    title = soup.title.text.replace("\n", "").replace("  ", "").replace(" – Carrucci Shoes", "").replace("Carrucci ", "")
    title = title.replace("Navy/Olive/Cognac", "").replace("-Black", "").replace("-Emerald", "").replace("PRE ORDER", "")
    title = title.replace("-Brown/Cognac", "").replace("-Brown/Navy", "").replace("-Navy/Red", "").replace("-Brown", "")
    title = title.replace("-Whisky", "").replace("-Ocean Blue", "").replace("-Cobalt", "").replace("-Gray", "")
    title = title.replace("-Wheat", "").replace("-Burgundy/Black", "").replace("-Burgundy", "").replace(" - Blue", "")
    title = title.title()
    
    # type
    type = "Dress Shoe"
    
    if "Oxford" in title:
        type = "Oxford"
    elif "Loafer" in title or "Slip-On" in title or "Slip On" in title or "Driver" in title:
        type = "Loafer and Slip On"
    elif "Boot" in title:
        type="Boot"
    elif "Sneaker" in title:
        type="Dress Casual Shoe"
    else:
        type="Dress Shoe"

    # seo title
    seotitle = 'Brite Creations | ' + title + ' by ' + vendor

    # body
    body = soup.find("div", {"class": "ProductMeta__Description"}).find("div", {"class": "Rte"})
    if body.find("ul") is None:
        body = body
    else:
        body = body.find("ul")
    body = str(body).replace("<div class=\"Rte\">", "").replace("</div>", "").replace("<span>", "").replace("</span>", "")
    body = "<meta charset=\"utf-8\">\n" + body + "\n<br><p>Best quality and price from <a href=\"https://britecreations.com/pages/about\">Brite Creations Atlanta</a>.</p>"
    
#     seodescription
    seodescription = 'Shop at Brite Creations Atlanta for the best deal on ' + title + " " + type +' by ' + vendor + '\n' + body
    
#     images
    imgs = soup.find_all("img")
    images = ["https:" + i['data-src'].replace("_{width}x", "") for i in imgs if "data-widths" in str(i)]
    
    colors = soup.find("div", {"class":"ProductForm__Variants"})
    colors = colors.find_all("input", {"class" : "ColorSwatch__Radio"})
    colors = [c["value"] for c in colors]
    
    sizes = soup.find("div", {"class":"ProductForm__Variants"})
    sizes = sizes.find_all("input", {"class" : "SizeSwatch__Radio"})
    sizes = [c["value"] for c in sizes]
    
#     print(sizes)
    
    
    
    if len(colors) == 0:
        if "KS525-102P" in soup.title.text:
            colors = ['Black']
        if "KS500-12" in soup.title.text:
            colors = ["Cognac"]
        if "KS524-12" in soup.title.text:
            colors = sizes[:4]
            sizes = sizes[4:]
        if "KS886-734" in soup.title.text:
            colors = sizes[:2]
            sizes = sizes[2:]
    if "KB2019-11" in soup.title.text:
        colors[0] = "Brown"
            
    fcolors = []
    for i in images:
        for c in colors:
            if c.replace("/", "").split()[0] in i or c.replace("/", "_").split()[0] in i or c.replace("/", " ").split()[0] in i:
                if c not in [a for a, b in fcolors]:
                    fcolors.append((c, i))
                break
    
    if len(fcolors) == 0:
        if len(images) == 1:
            for c in colors:
                fcolors.append((c, images[0]))
        if len(images) == len(colors):
            for c in range(len(colors)):
                fcolors.append((colors[c], images[c]))
        if len(colors) == 1:
            fcolors.append((colors[0], images[0]))
        if "KS503-37A" in soup.title.text:
            i = 2
            for c in colors:
                fcolors.append((c, images[i]))
                i += 1
        if "KS673-01" in soup.title.text:
            i = 1
            for c in colors:
                fcolors.append((c, images[i - 1]))
                i *= 2
                
    if len(colors) != len(fcolors):
        if len(images) == len(colors):
            fcolors = []
            for c in range(len(colors)):
                fcolors.append((colors[c], images[c]))

    if len(colors) != len(fcolors) and len(fcolors) != len(images):
        
        print(soup.title.text)
        print(fcolors)
        print(colors)
        print(images)
        print()
        print()


      KS503-02 Carrucci Signature Buckle Loafer – Carrucci Shoes
    
[('Chestnut', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS503-02Chestnut6.jpg?v=1604265236'), ('Red', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS478-02_Red2_e9425efc-2fc3-497a-a46d-73f1f210f60f.JPG?v=1594872463'), ('White', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS503-02_White2_f9fee8f8-f7a1-489c-9c78-ce93542a87ee.JPG?v=1594872463'), ('Purple', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS478-02_Purple2_4b99160d-9782-484b-b951-865a18ede746.JPG?v=1594872463'), ('Olive', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS478-02_Olive2_d58e7cf2-f412-4a14-95cb-ac4ef3efde79.JPG?v=1594872463'), ('Black', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS503-02Black5_cac72e40-eadf-4783-abe9-1c817ebe050a.jpg?v=1594872463')]
['Chestnut', 'Navy', 'Black', 'Cognac', 'Red', 'White', 'Purple', 'Olive', 'Gray']
['https://cdn.shopify.com/s/files/1/2027/1953/products/KS5


      KS509-14 Carrucci Cap Toe Leather Oxford – Carrucci Shoes
    
[('Black', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS509-14Black4.jpg?v=1604182105')]
['Black', 'navy', 'cognac']
['https://cdn.shopify.com/s/files/1/2027/1953/products/KS509-14Navy4.jpg?v=1604182105', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS509-14Black4.jpg?v=1604182105', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS509-14Cognac4.jpg?v=1604182105', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS509-14Navy2.jpg?v=1604021927', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS509-14Navy5.jpg?v=1604021927', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS509-14Black2.jpg?v=1604021927', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS509-14Cognac2.jpg?v=1604021928', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS509-14Outsole.jpg?v=1603935822']



      KS611-11 Carrucci Burnished Leather Sneaker – Carrucci Shoes
    
[('Cognac/Olive', 'htt


      KS671-01 Mid Top Leather Sneaker – Carrucci Shoes
    
[('Burgundy', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS671-01Burgundy4.jpg?v=1600056687'), ('Black', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS671-01Black4.jpg?v=1600056688'), ('Whisky', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS671-01Whisky4.jpg?v=1600056685'), ('Navy Patent Leather', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS671-01PNavy1.jpg?v=1600056687')]
['Burgundy', 'Black', 'Grey', 'Whisky', 'Black Patent Leather', 'Burgundy Patent Leather', 'Navy Patent Leather']
['https://cdn.shopify.com/s/files/1/2027/1953/products/KS671-01Burgundy4.jpg?v=1600056687', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS671-01Black4.jpg?v=1600056688', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS671-01Black2.jpg?v=1600056687', 'https://cdn.shopify.com/s/files/1/2027/1953/products/KS671-01Burgundy1.jpg?v=1600056685', 'https://cdn.shopify.com/s/files/1/2027/1953/p

In [3]:
def getType(a):
    if any(w in a.lower() for w in ['boot', 'belucci']):
        return 'Boot'
    if any(w in a.lower() for w in ['monk', 'sneaker', 'dress-casual']):
        return 'Dress Casual Shoe'
    if any(w in a.lower() for w in ['loafer', 'slip on', 'moccasin', 'mocassin', 'pump', 'porty', 'rollini']):
        return 'Loafer and Slip On'
    return 'Dress Shoe'

In [4]:
def saveSwatch(color, imglink):
    # finds and downloads swatches
    response = requests.get(imglink)
    try:
        image = Image.open(BytesIO(response.content))
    except:
        return
    x, y = image.size
    xl = x//2 - x//20
    xr = xl + x//10
    yl = 13*y//16 - y//20
    yr = yl + y//10
    image = image.crop((xl, yl, xr, yr))
    pt = "swatches/"+ (color.lower().replace(' ', '-').replace('/', '-'))

#     if path.exists(pt):
    image.save(pt + str(randrange(10)) +".png")
#     else:
#         image.save(pt +".png")

In [12]:
def getProduct(url, debug=False):
    res = requests.get(url)
    if res.status_code != 200:
        print("URL Error: " + url)
        return None
    soup = BeautifulSoup(res.content, "html.parser")
    
#     PRE-KNOWN INFORMATION
    vendor = "Mezlan"

    
#     title
    title = soup.title.text
    title = title.title()
    
#     handle
    handle = vendor.lower()+title.replace(' ', '').lower().replace("-", "").replace("/", "")
    
#     type
    ptype = soup.find("div", {"class": "std"}).find("ul").find_all("li")[0].text
    type = getType(str(title + " " + ptype))
    
#     seo title
    seotitle = 'Brite Creations | ' + title + " " + type +' by ' + vendor
    
#     body
    body = soup.find("div", {"class": "std"}).find("ul")
    body = str(body)
    body = "<meta charset=\"utf-8\">\n" + body[:-5] + "\n<li>Best quality and price from <a href=\"https://britecreations.com/pages/about\">Brite Creations Atlanta</a>.</li>\n</ul>"
    
#     seo description
    seodescription = 'Shop at Brite Creations Atlanta for the best deal on ' + title + " " + type +' by ' + vendor + '\n' + body

#     sizes, colors, images
    colsiz = soup.find_all("span", {"class": "swatch-label"})[:-1]
    sizes = [str(a.text.replace(' ', '').replace('\n', '').replace('½', '.5')).replace('.0', '') for a in colsiz if a.text.replace('\n', '') != '' and not ('A' in str(a) or 'E' in str(a) or 'I' in str(a) or 'O' in str(a) or 'U' in str(a))]
    try:
        colors = [a.find("img")['alt'].title() for a in colsiz if ('A' in str(a) or 'E' in str(a) or 'I' in str(a) or 'O' in str(a) or 'U' in str(a))]
        colors = [c for c in colors if c != '']
    except:
        colors = [str(a.text.replace(' ', '').replace('\n', '')).title() for a in colsiz if ('A' in str(a) or 'E' in str(a) or 'I' in str(a) or 'O' in str(a) or 'U' in str(a))]
        colors = [c for c in colors if c != '']
    images = soup.find_all("img", {"class": "gallery-image"})
    images = [a['src'] for a in images if '1800x' in a['src']]
    
    if debug: 
        print(colors)
        inp = input()
        if inp == "a":
            colors.append(input())
        if inp == "r":
            index = input()
            colors[int(index)] = input()
        print(colors)
    
    
    finalcolors = []
    finalimages = []
    
    for i in images:
        rep = "XXXXX"
        for j in colors:
            if j.lower().replace('/', '-').replace(' ', '-') in i:
                rep = j.lower().replace('/', '-').replace(' ', '-')
                break
        i = i.replace(rep, 'XXXXX')
        for c in colors:
            img = i.replace('XXXXX', c.lower().replace('/', '-').replace(' ', '-'))
            try:
                img = img[:img.index("/cache/")] + img[img.index("/m/e/"):] 
            except:
                img = img[:img.index("/cache/")] + img[img.index("/b/a/"):] 
            res = requests.get(img)
            if c not in [cl for cl, im in finalcolors]:
                finalcolors.append((c, img))
            if res.status_code == 200:
                if img not in [im for im, at in finalimages]:
                    finalimages.append((img, ''))
                else:
                    print("error", img)
            else:
                img = img.replace("_1", "").replace("_5", "").replace("_4", "").replace("_2", "").replace("_3", "").replace("_6", "").replace("_7", "")
                res = requests.get(img)
                if res.status_code == 200:
                    if img not in [im for im, at in finalimages]:
                        finalimages.append((img, ''))
                    else:
                        print("error", img)

    colors = finalcolors
    images = finalimages
    
#     tags
    tags = []
    for c, i in colors:
        tag = 'color:' + c
        if tag not in tags:
            tags.append(tag)
    for s in sizes:
        tag = 'size:' + str(s).replace('.0', '')
        if tag not in tags:
            tags.append(tag)

#     price
    p = float(soup.find("span", {"class": "price"}).text.replace(',', '').replace('$', ''))
    op = p - 20
    
    price = []
    oldprice = []
    for i in range(len(colors) * len(sizes)):
        price.append(op)
        oldprice.append(p)
    
#     inventory and sku
    inventory = []
    sku = []
    
    for i in range(len(colors) * len(sizes)):
        inventory.append((1, 0)) #(online, retail)
    
    for s in sizes:
        for c, i in colors:
            sku.append(handle+"-"+c.lower()+"-"+str(s).replace('.0', ''))
    
    swatches
    for c, i in colors:
        saveSwatch(c, i)
    
    prod = product(handle=handle, title=title, body=body, vendor=vendor, type=type, sizes=sizes, 
                   colors=colors, images=images, tags=tags, price=price, oldprice=oldprice, seotitle=seotitle, 
                   seodescription=seodescription, sku=sku, inventory=inventory)
    return prod

In [18]:
products = []
i = 1
for link in np.array(links):
    p = getProduct(link)
    products.append(p)
    print(i, link)
    i += 1
print(len(products))

1 https://www.mezlan.com/7276-f
2 https://www.mezlan.com/9730
3 https://www.mezlan.com/9711
4 https://www.mezlan.com/9647
5 https://www.mezlan.com/9743
6 https://www.mezlan.com/9641
7 https://www.mezlan.com/9621
8 https://www.mezlan.com/9802
9 https://www.mezlan.com/9736
10 https://www.mezlan.com/9744
11 https://www.mezlan.com/9735
12 https://www.mezlan.com/9727
13 https://www.mezlan.com/4733-j
14 https://www.mezlan.com/still-water-176678
15 https://www.mezlan.com/gehry-1
16 https://www.mezlan.com/9726
17 https://www.mezlan.com/9712
18 https://www.mezlan.com/9620
19 https://www.mezlan.com/9661
20 https://www.mezlan.com/9616
21 https://www.mezlan.com/7310-s
22 https://www.mezlan.com/9480
23 https://www.mezlan.com/8724
24 https://www.mezlan.com/montes
25 https://www.mezlan.com/4684-sp
26 https://www.mezlan.com/4642-f
27 https://www.mezlan.com/9598
28 https://www.mezlan.com/9467
29 https://www.mezlan.com/9512
30 https://www.mezlan.com/4638-sp
31 https://www.mezlan.com/malcolm-169900
32 ht

In [14]:
for i in [23, 33]:
    products[i] = getProduct(links[i], debug=True)

['Grey/Burgundy', 'Taupe/Cognac']
a
Cognac/Blue
['Grey/Burgundy', 'Taupe/Cognac', 'Cognac/Blue']
['Honey/Cognac', 'Mediumblue/Navy']
r
1
Medium Blue/Navy
['Honey/Cognac', 'Medium Blue/Navy']


In [15]:
for i in range(91):
    r = True
    r = len(products[i].images) >= len(products[i].colors)*3
    r = r and (len(products[i].images) <= len(products[i].colors)*5)
    print(i, r)

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True
19 True
20 True
21 True
22 True
23 True
24 True
25 True
26 True
27 True
28 True
29 True
30 True
31 True
32 True
33 True
34 True
35 True
36 True
37 True
38 True
39 True
40 True
41 True
42 True
43 True
44 True
45 True
46 True
47 True
48 True
49 True
50 True
51 True
52 True
53 True
54 True
55 True
56 True
57 True
58 True
59 True
60 True
61 True
62 True
63 True
64 True
65 True
66 True
67 True
68 True
69 True
70 True
71 True
72 True
73 True
74 True
75 True
76 True
77 True
78 True
79 True
80 True
81 True
82 True
83 True
84 True
85 True
86 True
87 True
88 True
89 True
90 True


In [17]:
productstocsv(products)
productstoinventory(products)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
Errors: []


In [20]:
print(products[62])

Bardem
----------------------------------
Handle: 	mezlanbardem
Sizes: 		['8.5', '9', '9.5', '10', '10.5', '11', '11.5', '12', '13']
Colors: 	[('Burgundy/Black', 'https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-8979-bardem-burgundy-black-1.1604171107.jpg'), ('Electric Blue/Blue', 'https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-8979-bardem-electric-blue-blue-1.1604171107.jpg'), ('Tan/Brown', 'https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-8979-bardem-tan-brown-1.1604171107.jpg')]
SKU: 		['mezlanbardem-burgundy/black-8.5', 'mezlanbardem-electric blue/blue-8.5', 'mezlanbardem-tan/brown-8.5', 'mezlanbardem-burgundy/black-9', 'mezlanbardem-electric blue/blue-9', 'mezlanbardem-tan/brown-9', 'mezlanbardem-burgundy/black-9.5', 'mezlanbardem-electric blue/blue-9.5', 'mezlanbardem-tan/brown-9.5', 'mezlanbardem-burgundy/black-10', 'mezlanbardem-electric blue/blue-10', 'mezlanbardem-tan/brown-10', 'mezlanbardem-burgundy/black-10.5', 'mezlanbardem-electri